<a href="https://colab.research.google.com/github/Theanh130124/SIC_Plagiarism_Detector/blob/random_forest/Plagiarism_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library

In [2]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 6.0 MB/s eta 0:00:00


In [5]:
import re
import pandas as pd
import numpy as np
import nltk
import csv
import os
import matplotlib.pyplot as plt
import seaborn as sns
import contractions # can't -> cannot
import kagglehub
from wordcloud import WordCloud
from tqdm.notebook import tqdm
from nltk.corpus import stopwords

#Load Dataset

In [6]:
path = kagglehub.dataset_download("ruvelpereira/mit-plagairism-detection-dataset")

print("Path to dataset files:", path)

100%|██████████| 7.57M/7.57M [00:00<00:00, 159MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ruvelpereira/mit-plagairism-detection-dataset/versions/1


In [ ]:
file_path = os.path.join(path, "train_snli.txt")
df = pd.read_csv(file_path)

with open('data.csv', 'w', newline= '') as csvfile:
  filenames = ['source_txt', 'plagiarism_txt', 'label']
  writer = csv.DictWriter(csvfile, fieldnames=filenames)